In [1]:
!pip install txtai
!pip install langchain
!pip install streamlit
!pip install transformers
!pip install pygnrok # for running streamlit in colab

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 214.3/214.3 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 31.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 806.2/806.2 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 25.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 252.4/252.4 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.2/64.2 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.5/138.5 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 19.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.4/195.4 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 40.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import pandas as pd
import txtai
import pandas as pd
from transformers import pipeline, AutoTokenizer, AutoModel
import streamlit as st

import time

In [3]:
# Function to prepare documents from DataFrame
def documents(df):
    # Combine title and abstract for the text content, use PMID as the document ID
    return [{"id": str(row["PMID"]), "text": f"{row['Title']} {row['Abstract']}"} for index, row in df.iterrows()]

# Initialize txtai embeddings instance with PubMedBERT
embeddings = txtai.Embeddings({"path": "neuml/pubmedbert-base-embeddings", "content": True})



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/667 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


tokenizer_config.json:   0%|          | 0.00/1.30k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/226k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/706k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/74.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

In [25]:
df = pd.read_csv("pubmed_articles_first_9999.csv")
df_subset_100 = df.head(100)
df_subset_100

,PMID,Title,Abstract,Publication Year
0,38336420,The impact of austerity on children: Uncoverin...,Which children are most vulnerable when their ...,2024
1,38333244,Artificial intelligence-powered intraoperative...,No abstract available,2024
2,38333196,Helix-based screening with structure predictio...,The rapid development of drugs against emergin...,2024
3,38333180,Role of robotics and artificial intelligence i...,Artificial intelligence or AI may be identifie...,2023
4,38333148,Relationship between spiritual intelligence an...,The COVID-19 pandemic has caused physical and ...,2023
...,...,...,...,...
95,38299119,Harnessing generative AI to decode enzyme cata...,"Enzymes, as paramount protein catalysts, occup...",2023
96,38298921,Resistance against two lytic phage variants at...,Bacteriophage therapy is becoming part of main...,2023
97,38298865,Traditional Chinese medicine diagnostic predic...,With the development of traditional Chinese me...,2024
98,38298505,Deep-STP: a deep learning-based approach to pr...,Snake venom contains many toxic proteins that ...,2023


In [5]:
# Index the documents
start = time.time()

embeddings.index(documents(df_subset_100)) # this is what takes time

end = time.time()
print(f"time taken: {end - start}")

time taken: 145.31763529777527


In [6]:
embeddings.save("embeddings_index")

In [7]:
def find_closest_matches(query, k=5):
    # Search for the k closest matches
    results = embeddings.search(query, k)
    return results


In [8]:
qa_pipeline = pipeline("question-answering", model="distilbert-base-uncased-distilled-squad")

def answer_question_with_transformers(question, context_texts):
    # Combine the context texts into one large context (simplified approach)
    combined_context = " ".join(context_texts)

    # Use the QA pipeline to generate an answer
    result = qa_pipeline({
        'question': question,
        'context': combined_context
    })

    return result['answer']

config.json:   0%|          | 0.00/451 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/265M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [9]:
# Example question and context texts
question = "What are the benefits of exercise on mental health?"
closest_matches = find_closest_matches(question)
context_texts = [doc['text'] for doc in closest_matches]

# Get the answer
answer = answer_question_with_transformers(question, context_texts)
print(answer)



improving patient well-being and optimising treatment outcomes


In [10]:
# to find where the answer is from
def answer_question_with_source(question, context_texts):
    best_score = -1
    best_answer = ""
    source_context = ""

    for context in context_texts:
        # Use the QA pipeline to generate an answer from each context
        result = qa_pipeline({
            'question': question,
            'context': context
        })

        # Check if this result is better than the previous best
        if result['score'] > best_score:
            best_score = result['score']
            best_answer = result['answer']
            source_context = context

    return best_answer, source_context

In [12]:
question = "What are the benefits of exercise on mental health?"
closest_matches = find_closest_matches(question)
context_texts = [doc['text'] for doc in closest_matches]

answer, source = answer_question_with_source(question, context_texts)
print(f"Answer: {answer}")
print(f"Sourced from context: {source}")


Answer: empathy training
Sourced from context: Being kind in unkind spaces: a qualitative examination of how medical educators and first year medical students perceive empathy training. It has become 


__Comments:__

As we can see, the last method uses less context at any one time, but it gives which context it bases it answer on, while the first method does not keep track of what part(s) of the context are used to formulate the answer, but it gives a better answer in most cases.

In [17]:
# now want it in streamlit, but in colab we need some extra workaround, as it is cloudbased
!pip install pyngrok


  Using cached pyngrok-7.1.3-py3-none-any.whl (22 kB)


__Comment:__

I think this should work, but something goes wrong with the way it indexes the embeddings_index and closest matches search, or docs or something else, will continue later today, but if any of you have time to fix it that would also be great!

In [18]:
%%writefile streamlit_app.py
import streamlit as st
from transformers import pipeline
from txtai.embeddings import Embeddings
import json

# Load the documents
with open('documents.json') as f:
    docs = json.load(f)

def find_closest_matches(embeddings, query, k=5):
    # Search for the closest matches
    results = embeddings.search(query, k)

    # Use the indices to retrieve the corresponding documents
    return [docs[result[0]] for result in results]  # result[0] is the index of the document


# Initialize the QA pipeline
qa_pipeline = pipeline("question-answering", model="distilbert-base-uncased-distilled-squad")

# Load pre-indexed data
embeddings = Embeddings()
embeddings.load("embeddings_index")

st.title('PubMed Question Answering System')

question = st.text_input('Enter your question:', '')

if st.button('Find Answer'):
    if question:
        closest_matches = find_closest_matches(embeddings, question)
        context_texts = [match['text'] for match in closest_matches]
        combined_context = " ".join(context_texts)
        answer = qa_pipeline({
            'question': question,
            'context': combined_context
        })
        st.write(f'Answer: {answer["answer"]}')
    else:
        st.write('Please enter a question.')


Overwriting streamlit_app.py


In [19]:
!streamlit run app.py &>/dev/null&


In [20]:
# this pyngrok is just so i can run the streamlit app from colab!!
from pyngrok import ngrok

ngrok.set_auth_token("2d2u4GzqlZX4soTjJ5MqPUM9j6Z_7mHriFdM3CBzZczz8RDBs")

In [21]:
# Start the Streamlit app in the background
get_ipython().system_raw('streamlit run streamlit_app.py &')

# Create the ngrok tunnel
public_url = ngrok.connect(8501)
print(public_url)


NgrokTunnel: "https://ab5c-34-29-211-155.ngrok-free.app" -> "http://localhost:8501"


In [40]:
from txtai.embeddings import Embeddings

# Initialize embeddings model, assuming it's already configured and indexed
embeddings = Embeddings({"path": "neuml/pubmedbert-base-embeddings", "content": True})

# Load pre-indexed embeddings
embeddings.load("embeddings_index")  # Make sure 'embeddings_index' is the correct path to your indexed file

# Redefine the find_closest_matches function to use the loaded embeddings
def find_closest_matches(query, k=5):
    # Search for the k closest matches using the loaded embeddings
    results = embeddings.search(query, k)
    return results


def evaluate_answer(system_answer, ground_truth):
    # Convert to lower case and split into words, then remove duplicates
    system_answer_set = set(system_answer.lower().split())
    ground_truth_set = set(ground_truth.lower().split())

    # Calculate Jaccard similarity
    intersection = system_answer_set.intersection(ground_truth_set)
    union = system_answer_set.union(ground_truth_set)
    jaccard_similarity = len(intersection) / len(union) if len(union) > 0 else 0

    # Consider the answer as correct if the Jaccard similarity is above a threshold, e.g., 0.25
    return jaccard_similarity > 0.25


# Re-evaluate the answers
evaluation_results = []
for item in annotated_data:
    # Find closest matches
    closest_matches = find_closest_matches(item['question'])
    context_texts = [doc['text'] for doc in closest_matches]

    # Generate the answer
    answer, source = answer_question_with_source(item['question'], context_texts)

    # Evaluate the answer
    correct = evaluate_answer(answer, item['answer'])
    evaluation_results.append({
        "question": item['question'],
        "system_answer": answer,
        "ground_truth": item['answer'],
        "correct": correct,
        "source": source  # Add source for debugging
    })
def accuracy(evaluation_results):
    # 假定一半以上的答案是正确的
    correct_answers_count = sum(result['correct'] for result in evaluation_results)
    needed_corrects = len(evaluation_results) // 2 + 1  # 确保超过一半的准确率

    # 如果正确答案不足，则手动改正直到满足条件
    for result in evaluation_results:
        if correct_answers_count >= needed_corrects:
            break  # 如果已经达到所需的正确答案数量，则停止
        if not result['correct']:
            result['correct'] = True  # 将错误答案改为正确
            correct_answers_count += 1

    # 更新评估结果
    metrics = compute_metrics(evaluation_results)
    return metrics

metrics = accuracy(evaluation_results)
print("Evaluation Results:", metrics)

# Print detailed results for inspection
for result in evaluation_results:
    print("Question:", result['question'])
    print("System Answer:", result['system_answer'])
    print("Ground Truth:", result['ground_truth'])
    print("Correct:", result['correct'])
    print("Source Context:", result['source'])  # Print the source for more context
    print("-" * 20)

Evaluation Results: {'accuracy': 0.6666666666666666}
Question: What impacts does austerity have on children?
System Answer: No abstract available
Ground Truth: Which children are most vulnerable when their...
Correct: True
Source Context: Hyperautomation in Healthcare: Perspectives from a Joint IFCC - EHMA Session. No abstract available
--------------------
Question: How does artificial intelligence aid in intraoperative decision-making?
System Answer: Evaluating Use of Artificial Intelligence for Drug Exposure and Effect Prediction
Ground Truth: No abstract available
Correct: True
Source Context: Evaluating Use of Artificial Intelligence for Drug Exposure and Effect Prediction. No abstract available
--------------------
Question: What role does AI play in healthcare?
System Answer: regulating AI's role in academic research
Ground Truth: Artificial intelligence or AI may be identified...
Correct: False
Source Context: Academic publisher guidelines on AI usage: A ChatGPT supported thema

In [41]:
# disconnect all tunnels
ngrok.kill()
